In [1]:
import numpy as np
import phate

In [2]:
def make_swiss_roll(turns=2, a=0.1, base_delta_theta=0.1 * 2 * np.pi, theta=1 * np.pi, noise=0.0):
    thetas = []
    while theta < 2 * np.pi * turns:
        thetas.append(theta)
        # Adjust delta_theta to be inversely proportional to theta, but ensure it's never zero
        delta_theta = base_delta_theta / (1 + theta / (2 * np.pi))
        theta += delta_theta
    thetas = np.array(thetas)
    tmp_x = (a * thetas * np.cos(thetas))
    xmin, xmax = np.min(tmp_x), np.max(tmp_x)
    w = np.linspace(xmin, xmax, len(thetas))
    t, w = np.meshgrid(thetas, w)
    t = t.flatten()
    w = w.flatten()
    x = a * t * np.cos(t)
    y = a * t * np.sin(t)
    z = w
    noises_t = np.random.normal(0, noise * (t.max()-t.min()), len(t))
    noises_w = np.random.normal(0, noise * (w.max()-w.min()), len(w))
    t += noises_t
    w += noises_w
    return np.column_stack((x, y, z)), np.column_stack((t, w))


In [3]:
Xs, thetas = make_swiss_roll(turns=2, a=0.1, base_delta_theta=0.1 * 2 * np.pi, theta=1.5 * np.pi, noise=0.)
ambient_noise = np.random.normal(0, 0.05, size=Xs.shape)
Xs += ambient_noise

In [5]:
phate_op = phate.PHATE()
X_phate = phate_op.fit_transform(Xs)

Calculating PHATE...
  Running PHATE on 900 observations and 3 variables.
  Calculating graph and diffusion operator...
    Calculating KNN search...
    Calculating affinities...
  Calculated graph and diffusion operator in 0.02 seconds.
  Calculating optimal t...
    Automatically selected t = 29
  Calculated optimal t in 0.39 seconds.
  Calculating diffusion potential...
  Calculated diffusion potential in 0.26 seconds.
  Calculating metric MDS...
  Calculated metric MDS in 0.58 seconds.
Calculated PHATE in 1.27 seconds.


In [6]:
np.save(f"../data/swiss_roll.npy", Xs)
np.save(f"../data/swiss_roll_phate.npy", X_phate)